In [ ]:
## Part A: Write a script to play the data as a video
#Import all the libraries
import plotly.graph_objs as go
import tifffile
import numpy as np

# Function to read frames from a .tif file
def read_tif_frames(tif_path):
      # Use tifffile to open and read the tif file
    with tifffile.TiffFile(tif_path) as tif:
        # Assuming the image data is in the first page/series
        # You might need to adjust this depending on your TIFF file structure
        frames = tif.asarray()


    return frames

# Path to your .tif file
tif_path = "TEST_MOVIE_00001-small-motion.tif"

# Read the frames from the .tif file
frames = read_tif_frames(tif_path)

### Please note this line is for debugging, as loading the entire .tif takes quite a long time
### Please run this line to show a proof-of-concept of this video from frame 1 to 101.
frames = frames[0:200]

# Setup initial frame
fig = go.Figure(
    data=[go.Heatmap(z=frames[0])],
    layout=go.Layout(
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True}]),
                     dict(label="Pause",
                          method="animate",
                          args=[[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate"}])])],
        sliders=[{
            "steps": [{"args": [[f"frame{i}"], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate"}],
                       "label": str(i), "method": "animate"} for i in range(len(frames))]}]
    ),
    frames=[go.Frame(data=[go.Heatmap(z=frame)], name=f"frame{i}") for i, frame in enumerate(frames)]
)

# Show the figure
fig.show()

In [ ]:
## Part B: Randomly pick two frames that seem to be particularly offset from each other

import numpy as np
from scipy.signal import correlate2d

def compute_shifted_correlation(frame1, frame2, max_shift):
    """
    Compute the correlation between two frames for different shifts.
    
    :param frame1: First frame as a 2D NumPy array.
    :param frame2: Second frame as a 2D NumPy array.
    :param max_shift: Maximum number of pixels to shift in any direction.
    :return: A 2D array of correlation coefficients for each shift.
    """
    # Initialize an array to hold the correlation results
    correlation_map = np.zeros((2*max_shift+1, 2*max_shift+1))
    
    # Loop over all possible shifts within the specified range
    for dy in range(-max_shift, max_shift+1):
        for dx in range(-max_shift, max_shift+1):
            # Shift frame2 relative to frame1
            shifted_frame2 = np.roll(frame2, shift=(dy, dx), axis=(0, 1))
            
            # Compute correlation coefficient for this shift
            correlation = correlate2d(frame1, shifted_frame2, mode='valid')
            
            # Store the correlation result
            correlation_map[dy+max_shift, dx+max_shift] = correlation
            
    return correlation_map

# Example usage
frame1 = frames[1]  # Your first frame data here
frame2 = frames[200]  # Your second frame data here
max_shift = 10  # Example maximum shift in pixels

correlation_map = compute_shifted_correlation(frame1, frame2, max_shift)

# To find the shift with the maximum correlation
max_correlation_value = np.max(correlation_map)
peak_idx = np.unravel_index(np.argmax(correlation_map), correlation_map.shape)
print(f"Maximum correlation value: {max_correlation_value}")
print(f"Location of the correlation peak (shift_y, shift_x): {peak_idx}")


#max_correlation_idx = np.unravel_index(np.argmax(correlation_map), correlation_map.shape)
#optimal_shift = (max_correlation_idx[0] - max_shift, max_correlation_idx[1] - max_shift)
#print(f"Optimal shift for maximum correlation: {optimal_shift}")